In [1]:
# Generic
import typing
import os
from pathlib import Path
import typing
import json

# Plotting
import matplotlib.pyplot as plt

# Numerical Computing
import numpy as np
import torch

# muutils
from zanj.zanj import ZANJ, ZANJ_GLOBAL_DEFAULTS

# Our Code
from muutils.nbutils.configure_notebook import configure_notebook
from maze_transformer.training.config import ConfigHolder, ZanjHookedTransformer, BaseGPTConfig, TrainConfig
from maze_dataset import MazeDataset, MazeDatasetConfig, MazeDatasetCollectionConfig
from maze_dataset.dataset.configs import MAZE_DATASET_CONFIGS
from maze_dataset.generation import LatticeMazeGenerators
from maze_transformer.training.train_model import TrainingResult, train_model
from maze_transformer.training.wandb_logger import WandbProject


In [2]:
# set global defaults for ZANJ
ZANJ_GLOBAL_DEFAULTS.external_array_threshold = 1024
ZANJ_GLOBAL_DEFAULTS.external_list_threshold = 1024

# paths
PATH_EXAMPLES: Path = Path("../examples/")
PATH_DATA: Path = Path("../data/")

# reproducibility and device
DEVICE = configure_notebook(seed=42, dark_mode=True)
print(f"{DEVICE = }")

DEVICE = device(type='cuda')


In [3]:
print(f"{list(MAZE_DATASET_CONFIGS.keys()) = }")

# if you want to specify a custom config, you can do so here
CFG_CUSTOM: ConfigHolder = ConfigHolder(
    name = "hallway-medium",
    dataset_cfg = MazeDatasetConfig(
		name="custom-hallway",
		grid_n=8,
		n_mazes=1_000_000,
		maze_ctor=LatticeMazeGenerators.gen_dfs,
        maze_ctor_kwargs=dict(
            do_forks=False,
        ),
        applied_filters=[{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}],
	),
    model_cfg = BaseGPTConfig(
        name="custom-model",
        act_fn="gelu",
        d_model=128,
        d_head=32,
        n_layers=6,
    ),
    train_cfg = TrainConfig(
        name="custom-train",
        optimizer=torch.optim.AdamW,
        optimizer_kwargs=dict(lr=0.001),
        batch_size=32,
        dataloader_cfg=dict(
            shuffle=False,
            num_workers=8,
            drop_last=False,
        ),
        intervals_count=dict(
            print_loss=100,
            checkpoint=20,
            eval_fast=100,
            eval_slow=50,
        ),
        validation_dataset_cfg=100,
    ),
)

CFG_TEST: ConfigHolder = ConfigHolder(
    name = "hallway-nano",
    dataset_cfg = MazeDatasetConfig(
        name=f"custom-hallway",
        grid_n=3,
        n_mazes=50,
        maze_ctor=LatticeMazeGenerators.gen_dfs,
        maze_ctor_kwargs=dict(
            do_forks=False,
        ),
        applied_filters=[{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}],
    ),
    # TODO: fix this
    # dataset_cfg = MazeDatasetCollectionConfig(
    #     name="custom-hallway-collection",
    #     maze_dataset_configs=[
    #         MazeDatasetConfig(
    #             name=f"custom-hallway-{n}",
    #             grid_n=n,
    #             n_mazes=50,
    #             maze_ctor=LatticeMazeGenerators.gen_dfs,
    #             maze_ctor_kwargs=dict(
    #                 do_forks=False,
    #             ),
    #             applied_filters=[{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}],
    #         ) 
    #         for n in [3, 4]
    #     ]
    # ),
    model_cfg = BaseGPTConfig(
        name="custom-model",
        act_fn="gelu",
        d_model=8,
        d_head=2,
        n_layers=2,
    ),
    train_cfg = TrainConfig(
        name="custom-train",
        optimizer=torch.optim.AdamW,
        optimizer_kwargs=dict(lr=0.0001),
        batch_size=4,
        dataloader_cfg=dict(
            shuffle=True,
            drop_last=False,
        ),
       validation_dataset_cfg=5,
    ),
)

list(MAZE_DATASET_CONFIGS.keys()) = ['test-g3-n5-a_dfs-h89001', 'demo_small-g3-n100-a_dfs-h58410', 'demo-g6-n10K-a_dfs-h86254']


In [4]:
# here is where to specify which config to actually use
CFG: ConfigHolder = CFG_CUSTOM # change to CFG_CUSTOM to train a "real" model, the CFG_TEST is for CI testing

In [5]:
print(json.dumps(CFG.summary(), indent=2))

{
  "name": "hallway-medium",
  "dataset_cfg": {
    "name": "custom-hallway",
    "fname": "custom-hallway-g8-n1000000-a_dfs-h67201",
    "sdc_hash": 17702762527462501104362081349751668865096610287696345990139527306009298467201,
    "seed": 42,
    "seq_len_min": 1,
    "seq_len_max": 512,
    "padding_token_index": 10,
    "token_arr_joined": "<ADJLIST_START> <ADJLIST_END> <TARGET_START> <TARGET_END> <ORIGIN_START> <ORIGIN_END> <PATH_START> <PATH_END> <--> ; <PADDING> (0,0) (0,1) (1,0) (1,1) (0,2) (2,0) (1,2) (2,1) (2,2) (0,3) (3,0) (3,1) (2,3) (3,2) (1,3) (3,3) (0,4) (2,4) (4,0) (1,4) (4,1) (4,2) (3,4) (4,3) (4,4) (0,5) (5,0) (5,1) (2,5) (5,2) (5,3) (4,5) (5,4) (1,5) (3,5) (5,5) (0,6) (2,6) (4,6) (6,0) (1,6) (6,1) (6,2) (3,6) (6,3) (6,4) (5,6) (6,5) (6,6) (0,7) (7,0) (7,1) (2,7) (7,2) (7,3) (4,7) (7,4) (7,5) (6,7) (7,6) (1,7) (3,7) (5,7) (7,7)",
    "applied_filters": [
      {
        "name": "collect_generation_meta",
        "args": [],
        "kwargs": {}
      }
    ],
    "gr

In [6]:
# get just the dataset, generating it if needed. 
# This step can be skipped if you set `do_generate_dataset=True` when calling `train_model`
# or if the dataset in question already exists

# load the dataset
DATASET: MazeDataset = MazeDataset.from_config(
    CFG.dataset_cfg, 
    verbose=True, 
    load_local=False,
    save_local=True,
    local_base_path=PATH_DATA,
)
CFG.dataset_cfg = DATASET.cfg

seeing if we can download the dataset...
no download found, or download failed
generating dataset...


generating & solving mazes: 100%|██████████| 1000000/1000000 [21:12<00:00, 786.11maze/s]


saving dataset to ..\data\custom-hallway-g8-n1000000-a_dfs-h67201.zanj
Got dataset custom-hallway with 1000000 items. output.cfg.to_fname() = 'custom-hallway-g8-n1000000-a_dfs-h67201'


In [7]:
result: TrainingResult = train_model(
	base_path=PATH_DATA,
    cfg=CFG,
	wandb_project=WandbProject.DEMO_NOTEBOOKS, # change this to WandbProject.DEMO_NOTEBOOKS! INTEGRATION_TESTS is for CI testing
	do_generate_dataset=False,
	dataset_verbose=True,
)

2023-06-16 03:40:47 ERROR Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: miv. Use `wandb login --relogin` to force relogin


2023-06-16 03:40:50 INFO config ={'__format__': 'ConfigHolder(SerializableDataclass)', 'dataset_cfg': {'__format__': 'MazeDatasetConfig(SerializableDataclass)', 'name': 'custom-hallway', 'seq_len_min': 1, 'seq_len_max': 512, 'seed': 42, 'applied_filters': [{'name': 'collect_generation_meta', 'args': (), 'kwargs': {}}], 'grid_n': 8, 'n_mazes': 1000000, 'maze_ctor': {'__name__': 'gen_dfs', '__module__': 'maze_dataset.generation.generators', '__doc__': ['generate a lattice maze using depth first search, iterative', '', '        # Arguments', '        - `grid_shape: Coord`: the shape of the grid', '        - `lattice_dim: int`: the dimension of the lattice', '          (default: `2`)', '        - `n_accessible_cells: int | None`: the number of accessible cells in the maze. If `None`, defaults to the total number of cells in the grid.', '            (default: `None`)', '        - `max_tree_depth: int | None`: the maximum depth of the tree. If `None`, defaults to `2 * n_accessible_cells`.', 

F:\KNC\maze-transformer\maze_transformer\evaluation\path_evals.py:91: RuntimeWarning:

fraction_connections_adjacent_lattice called on path of length less than 2, retuning NaN
prediction = array([[4, 1]])



2023-06-16 03:47:34 INFO Running evals: eval_slow
2023-06-16 03:47:48 INFO iteration 0/31247: loss=4.777
2023-06-16 03:47:48 INFO Saving model checkpoint to ../data/hallway-medium_2023-06-16-03-40-47/checkpoints/model.iter_0.zanj
2023-06-16 03:48:38 INFO Running evals: eval_fast
2023-06-16 03:48:52 INFO iteration 312/31247: loss=0.988
2023-06-16 03:49:42 INFO Running evals: eval_fast
2023-06-16 03:49:55 INFO Running evals: eval_slow
2023-06-16 03:50:09 INFO iteration 624/31247: loss=0.698
2023-06-16 03:50:59 INFO Running evals: eval_fast


...

2023-06-16 04:58:19 INFO Running evals: eval_slow
2023-06-16 04:58:31 INFO iteration 16224/31247: loss=0.529
2023-06-16 04:59:36 INFO Running evals: eval_fast


F:\KNC\maze-transformer\maze_transformer\evaluation\path_evals.py:91: RuntimeWarning:

fraction_connections_adjacent_lattice called on path of length less than 2, retuning NaN
prediction = array([[1, 3]])



2023-06-16 04:59:48 INFO iteration 16536/31247: loss=0.585
2023-06-16 05:00:57 INFO Running evals: eval_fast
2023-06-16 05:01:09 INFO Running evals: eval_slow
2023-06-16 05:01:22 INFO iteration 16848/31247: loss=0.558
2023-06-16 05:02:33 INFO Running evals: eval_fast
2023-06-16 05:02:45 INFO iteration 17160/31247: loss=0.445
2023-06-16 05:02:48 INFO Saving model checkpoint to ../data/hallway-medium_2023-06-16-03-40-47/checkpoints/model.iter_17182.zanj
2023-06-16 05:03:55 INFO Running evals: eval_fast
2023-06-16 05:04:07 INFO Running evals: eval_slow
2023-06-16 05:04:19 INFO iteration 17472/31247: loss=0.389
2023-06-16 05:05:25 INFO Running evals: eval_fast
2023-06-16 05:05:38 INFO iteration 17784/31247: loss=0.481
2023-06-16 05:06:50 INFO Running evals: eval_fast
2023-06-16 05:07:03 INFO Running evals: eval_slow
2023-06-16 05:07:15 INFO iteration 18096/31247: loss=0.474
2023-06-16 05:08:24 INFO Running evals: eval_fast
2023-06-16 05:08:36 INFO iteration 18408/31247: loss=0.592
2023-06-

F:\KNC\maze-transformer\maze_transformer\evaluation\path_evals.py:91: RuntimeWarning:

fraction_connections_adjacent_lattice called on path of length less than 2, retuning NaN
prediction = array([[2, 1]])



2023-06-16 05:21:30 INFO Running evals: eval_slow
2023-06-16 05:21:42 INFO iteration 21216/31247: loss=0.564
2023-06-16 05:22:50 INFO Running evals: eval_fast
2023-06-16 05:23:02 INFO iteration 21528/31247: loss=0.590
2023-06-16 05:24:15 INFO Running evals: eval_fast
2023-06-16 05:24:27 INFO Running evals: eval_slow
2023-06-16 05:24:39 INFO iteration 21840/31247: loss=0.522
2023-06-16 05:24:44 INFO Saving model checkpoint to ../data/hallway-medium_2023-06-16-03-40-47/checkpoints/model.iter_21868.zanj
2023-06-16 05:25:49 INFO Running evals: eval_fast
2023-06-16 05:26:02 INFO iteration 22152/31247: loss=0.661
2023-06-16 05:27:09 INFO Running evals: eval_fast
2023-06-16 05:27:22 INFO Running evals: eval_slow
2023-06-16 05:27:35 INFO iteration 22464/31247: loss=0.534
2023-06-16 05:28:45 INFO Running evals: eval_fast
2023-06-16 05:28:58 INFO iteration 22776/31247: loss=0.526
2023-06-16 05:30:06 INFO Running evals: eval_fast
2023-06-16 05:30:18 INFO Running evals: eval_slow
2023-06-16 05:30: